In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

import torchvision
import torchvision.utils
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor

In [ ]:
train_set = datasets.LFWPairs(
    root='data',
    split='train',
    transform=ToTensor(),
    download=True
)

test_set = datasets.LFWPairs(
    root='data',
    split='test',
    transform=ToTensor(),
    download=True
)

In [ ]:
len(train_set), len(test_set)

In [ ]:
# use 20% of training data for validation
train_set_size = int(len(train_set) * 0.8)
valid_set_size = len(train_set) - train_set_size

seed = torch.Generator().manual_seed(42)
train_set, valid_set = data.random_split(train_set, [train_set_size, valid_set_size], generator=seed)

In [ ]:
len(train_set), len(valid_set), len(test_set)

In [ ]:
n_samples = 5

for i in range(n_samples):
  n = 6
  m = np.reshape(np.linspace(0,1,n**2), (n,n))
  plt.figure(figsize=(14,3))

  plt.subplot(141)
  plt.imshow(np.transpose(train_set[i][0], (1, 2, 0)))
  plt.xticks(range(n))
  plt.yticks(range(n))

  plt.subplot(142)
  plt.imshow(np.transpose(train_set[i][1], (1, 2, 0)))
  plt.yticks([])
  plt.xticks(range(n))

plt.show()

In [ ]:
for i in range(n_samples):
  print(train_set[i][2])

In [ ]:
train_dataloader = DataLoader(train_set,
                          shuffle=True,
                          num_workers=2,
                          batch_size=64)

valid_dataloader = DataLoader(valid_set,
                          shuffle=True,
                          num_workers=2,
                          batch_size=64)

test_dataloader = DataLoader(test_set,
                         shuffle=True,
                         num_workers=2,
                         batch_size=64)

In [ ]:
batch = next(iter(train_dataloader))
len(batch), len(batch[0]), len(batch[1]), len(batch[2])

In [ ]:
class SiameseNetwork(nn.Module):

    def __init__(self):
        super(SiameseNetwork, self).__init__()

        self.cnn1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11,stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(256, 384, kernel_size=3,stride=1),
            nn.ReLU(inplace=True)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(38400, 1024),
            nn.ReLU(inplace=True),

            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),

            nn.Linear(256,2)
        )

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        return output1, output2

In [ ]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
      euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)

      loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                    (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

      return loss_contrastive

In [ ]:
siamese_net = SiameseNetwork().cuda()
loss_func = ContrastiveLoss()
opt = optim.Adam(siamese_net.parameters(), lr = 0.0003)

In [ ]:
counter = []
loss_history = []
itr= 0

for epoch in range(40):
    for i, (first_img, second_img, label) in enumerate(train_dataloader, 0):

        first_img, second_img, label = first_img.cuda(), second_img.cuda(), label.cuda()

        opt.zero_grad()

        output1, output2 = siamese_net(first_img, second_img)

        loss_contrastive = loss_func(output1, output2, label)
        loss_contrastive.backward()

        opt.step()

        if i % 100 == 0 :
            print(f"Epoch number {epoch+1}\nloss: {loss_contrastive.item():.2f}\n")
            itr += 10

            counter.append(itr)
            loss_history.append(loss_contrastive.item())

In [ ]:
plt.plot(counter, loss_history)
plt.show()

In [ ]:
def make_prediction(sum_dist, label, threshold):
  if sum_dist < threshold:
    prediction = 1
  else:
    prediction = 0

  return prediction

In [ ]:
TP = 0 # True Positive
TN = 0 # True Negative
FP = 0 # False Postive
FN = 0 # False Negative
cnt = 0


for i, (first_img, second_img, label) in enumerate(valid_dataloader, 0):
  batch_size = len(first_img)

  emb_vec1, emb_vec2 = siamese_net(first_img.cuda(), second_img.cuda())
  euclidean_distance = F.pairwise_distance(emb_vec1, emb_vec2)

  for j in range(batch_size):
    sum_euclidean = torch.sum(euclidean_distance[j])
    prediction = make_prediction(sum_euclidean, label[j], 0.8)

    if prediction == label[j]:
      if prediction == 1:
        TP += 1
      else:
        TN += 1

    else:
      if prediction == 1:
        FP += 1
      else:
        FN += 1


Accuracy = (TP+TN) / (TP+TN+FP+FN)
Precision = TP / (TP+FP)
Recall = TP / (TP+FN)
Specificity = TN / (TN+FP)
F1_Score = 2 * ((Precision*Recall)/(Precision+Recall))

In [ ]:
print(f"Accuracy: {Accuracy:.2f}")
print(f"Precision: {Precision:.2f}")
print(f"Recall: {Recall:.2f}")
print(f"Specificity: {Specificity:.2f}")
print(f"F1-Score: {F1_Score:.2f}")

In [ ]:
cosine_sim = nn.CosineSimilarity()

In [ ]:
TP = 0 # True Positive
TN = 0 # True Negative
FP = 0 # False Postive
FN = 0 # False Negative
cnt = 0


for i, (first_img, second_img, label) in enumerate(valid_dataloader, 0):
  batch_size = len(first_img)

  emb_vec1, emb_vec2 = siamese_net(first_img.cuda(), second_img.cuda())
  cosine_similiarity = cosine_sim(emb_vec1, emb_vec2)

  for j in range(batch_size):
    sum_cosine = torch.sum(cosine_similiarity[j])
    prediction = make_prediction(sum_cosine, label[j], 0.9)

    if prediction == label[j]:
      if prediction == 1:
        TP += 1
      else:
        TN += 1

    else:
      if prediction == 1:
        FP += 1
      else:
        FN += 1


Accuracy = (TP+TN) / (TP+TN+FP+FN)
Precision = TP / (TP+FP)
Recall = TP / (TP+FN)
Specificity = TN / (TN+FP)
F1_Score = 2 * ((Precision*Recall)/(Precision+Recall))

In [ ]:
print(f"Accuracy: {Accuracy:.2f}")
print(f"Precision: {Precision:.2f}")
print(f"Recall: {Recall:.2f}")
print(f"Specificity: {Specificity:.2f}")
print(f"F1-Score: {F1_Score:.2f}")

In [ ]:
first_img, second_img, label = next(iter(test_dataloader))

In [ ]:
plt.imshow(np.transpose(first_img[1], (1, 2, 0)))

In [ ]:
plt.imshow(np.transpose(second_img[1], (1, 2, 0)))

In [ ]:
emb_vec1, emb_vec2 = siamese_net(first_img.cuda(), second_img.cuda())
euclidean_distance = F.pairwise_distance(emb_vec1, emb_vec2)

sum_euclidean = torch.sum(euclidean_distance)
prediction = make_prediction(sum_euclidean, label[j], 0.8)

if prediction == 0:
  out = 'Different people'
else:
  out = 'Same people'

out